# I. US County Race Population

In [1]:
import pandas as pd
import numpy as np 
from datetime import datetime  


def diagnose_Rate(T, **kwargs):
    info_list = []
    for col in T.columns:
        if col in ['Rate_Black', 'Rate_White', 'Rate_Parity']:
            d = {'name': col}
            for k, v in kwargs.items():
                d[k] = v
                
            for k,v in T[col].describe().to_dict().items():
                d[k] = v
            info_list.append(d)

    Report = pd.DataFrame(info_list)
    return Report



In [2]:
# US_COUNTY_POPULATION[US_COUNTY_POPULATION['Total_Whole'] > 100000]

path = 'Data/US_COUNTY_POPULATION.csv'
US_COUNTY_POPULATION = pd.read_csv(path)
US_COUNTY_POPULATION.head()

,GEOID,State,County,Geographic Area Name,Total_Whole,Total_White,Total_Black,Total_AIndA,Total_Asian,Total_WhiteC,Total_BlackC,Total_AIndAC,Total_AsianC
0,1001,Alabama,Autauga,"Autauga, Alabama",55869,42607,11237,266,656,43563,11784,575,950
1,1003,Alabama,Baldwin,"Baldwin, Alabama",223234,195198,19593,1742,2380,199103,21354,3603,3351
2,1005,Alabama,Barbour,"Barbour, Alabama",24686,12134,11906,170,116,12380,12117,297,152
3,1007,Alabama,Bibb,"Bibb, Alabama",22394,17191,4763,103,48,17420,4907,207,97
4,1009,Alabama,Blount,"Blount, Alabama",57826,55412,978,370,185,56167,1276,820,312


In [3]:


STATE_VACCINE = {}


In [4]:
print("Enter the date of collecting data (Today's data in YYYY-MM-DD):")
Today_Date = input()

assert Today_Date == str(datetime.now().date())

print('Hello, ' + Today_Date)


Enter the date of collecting data (Today's data in YYYY-MM-DD):
2021-04-17
Hello, 2021-04-17


# II. API States

## A. Illinois* (Done)


https://www.dph.illinois.gov/covid19/vaccinedata?county=Illinois


* Only Number


* Admin-Count (People at least One)

* Full-Count (Fully Covered)


In [5]:
from datetime import datetime 
import numpy as np

import requests
import time



In [6]:
state = 'Illinois'


def get_Illinois_Raw_and_postprocess(US_COUNTY_POPULATION):
    

    il_county_list = [i for i in US_COUNTY_POPULATION[US_COUNTY_POPULATION['State'] == 'Illinois']['County'].values]

    il_county_list.sort()

    L = []
    error_list = []
    for c in il_county_list:
        # c = 'Mchenry'
        api_url = 'https://idph.illinois.gov/DPHPublicInformation/api/covidvaccine/getVaccineAdministrationDemos?countyname={}'.format(c)

        resp = requests.get(api_url)# .to_json()
        data = resp.json() # resp
        if sum([len(v) for k, v in data.items()]) == 0:
            error_list.append(c)
            print(c)
            continue

        time.sleep(0.1)
        try:
            D = {}
            D['County'] = c
            for d in data['Race']:
                D[d['Race'] + '-AdminCount'] = d['AdministeredCount']
                D[d['Race'] + '-FullyVaccn'] = d['PersonsFullyVaccinated']
                date = d['Report_Date']

            D['Date'] = date.split('T')[0]
            L.append(D)

        except:
            print('No Data')
            print(c)

    IL_County_Vaccine_Race = pd.DataFrame(L)
    
    return IL_County_Vaccine_Race




IL_County_Vaccine_Race = get_Illinois_Raw_and_postprocess(US_COUNTY_POPULATION)
# IL_County_Vaccine_Race.to_csv('FinalResult/Raw_Illinois.csv')
print(IL_County_Vaccine_Race.shape)
IL_County_Vaccine_Race.head()

(102, 18)


,County,Hispanic or Latino-AdminCount,Hispanic or Latino-FullyVaccn,Asian-AdminCount,Asian-FullyVaccn,Other race-AdminCount,Other race-FullyVaccn,Black or African-American-AdminCount,Black or African-American-FullyVaccn,White-AdminCount,White-FullyVaccn,American Indian or Alaska Nati-AdminCount,American Indian or Alaska Nati-FullyVaccn,Native Hawaiian or Other Pacif-AdminCount,Native Hawaiian or Other Pacif-FullyVaccn,Unknown-AdminCount,Unknown-FullyVaccn,Date
0,Adams,468,254,324.0,172.0,64,37,1054,582,38840,20693,52.0,27.0,52.0,25.0,779,378,2021-04-15
1,Alexander,8,3,NaN,NaN,7,3,428,188,761,337,5.0,1.0,2.0,1.0,96,46,2021-04-15
2,Bond,121,39,29.0,10.0,87,63,131,57,7780,3526,NaN,NaN,9.0,3.0,247,106,2021-04-15
3,Boone,3197,1025,339.0,132.0,383,157,406,151,22773,9582,108.0,49.0,35.0,14.0,1204,420,2021-04-15
4,Brown,10,5,6.0,3.0,1,1,19,10,2515,1203,6.0,3.0,NaN,NaN,55,25,2021-04-15


In [7]:


def standardize_Illinois(IL_County_Vaccine_Race, US_COUNTY_POPULATION):
    
    state = 'Illinois'

    new_L = []


    for idx, row in IL_County_Vaccine_Race.iterrows(): # change this
        d = row.to_dict()

        new_d = {}


        # GEO Information
        new_d['State'] = state
        new_d['County'] = d['County']


        RenameCounties = {
            # 'Mckean': 'McKean'
        }
        if new_d['County'] in RenameCounties:
            new_d['County'] = RenameCounties[new_d['County']]

        new_d['Geographic Area Name'] = new_d['County'] + ', ' + new_d['State']



        # At Least One
        new_d['Numb_White'] = d['White-AdminCount']
        new_d['Rate_White'] = np.nan

        new_d['Numb_Black'] = d['Black or African-American-AdminCount']
        new_d['Rate_Black'] = np.nan

        new_d['Numb_Asian'] = d['Asian-AdminCount']
        new_d['Rate_Asian'] = np.nan

        new_d['Numb_AIndA'] = d['American Indian or Alaska Nati-AdminCount']
        new_d['Rate_AIndA'] = np.nan

        new_d['Numb_Other'] = d['Other race-AdminCount']
        
       

        new_d['Numb_Hispa'] = d['Hispanic or Latino-AdminCount']
        

        new_d['Date'] = str(datetime.now().date())

        new_L.append(new_d)

    
    Data = pd.DataFrame(new_L)
    
    not_included = list(set(Data['Geographic Area Name'].unique()) - set(US_COUNTY_POPULATION['Geographic Area Name'].unique()))
    if len(not_included) > 0:
        print('Not included Counties', not_included)


    T = pd.merge(Data, US_COUNTY_POPULATION[US_COUNTY_POPULATION.columns[3:]], on = 'Geographic Area Name')


    for Race in ['White', 'Black', 'Asian', 'AIndA']:
        Rate = T['Numb_' + Race] / T['Total_' + Race]
        Rate = Rate.replace([-np.inf, np.inf], np.nan)
        T['Rate_' + Race] = Rate


        Rate = T['Numb_' + Race] / T['Total_' + Race + 'C']
        Rate = Rate.replace([-np.inf, np.inf], np.nan)
        T['Rate_' + Race + 'C'] = Rate

        
    T = T[list(T.columns[:3]) + list(T.columns[3:].sort_values())]
    return T
        
        


T = standardize_Illinois(IL_County_Vaccine_Race, US_COUNTY_POPULATION)
# T.to_csv('FinalResult/T_' + state +'.csv', index = False)
print(T.shape)
T.head()

(102, 27)


,State,County,Geographic Area Name,Date,Numb_AIndA,Numb_Asian,Numb_Black,Numb_Hispa,Numb_Other,Numb_White,...,Rate_WhiteC,Total_AIndA,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole
0,Illinois,Adams,"Adams, Illinois",2021-04-17,52.0,324.0,1054,468,64,38840,...,0.626543,190,395,615,830,2557,3554,60698,61991,65435
1,Illinois,Alexander,"Alexander, Illinois",2021-04-17,5.0,NaN,428,8,7,761,...,0.196438,26,62,17,36,1830,1917,3740,3874,5761
2,Illinois,Bond,"Bond, Illinois",2021-04-17,NaN,29.0,131,121,87,7780,...,0.514789,110,183,134,187,1041,1195,14874,15113,16426
3,Illinois,Boone,"Boone, Illinois",2021-04-17,108.0,339.0,406,3197,383,22773,...,0.449065,420,1006,785,1052,1503,1984,49530,50712,53544
4,Illinois,Brown,"Brown, Illinois",2021-04-17,6.0,6.0,19,10,1,2515,...,0.482540,87,107,21,43,1242,1276,5165,5212,6578


In [8]:
T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
STATE_VACCINE[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

['Illinois']


,name,count,mean,std,min,25%,50%,75%,max
0,Rate_Black,102.0,0.212857,0.124697,0.014374,0.127399,0.208839,0.269109,0.977477
1,Rate_White,102.0,0.511328,0.093473,0.203476,0.453597,0.510570,0.578509,0.773547
2,Rate_Parity,102.0,0.298471,0.135639,-0.401442,0.241347,0.313593,0.392486,0.540650


# III. Table States

## B. Texas* (Done)

download from: 
https://www.dshs.texas.gov/immunize/covid19/COVID-19-Vaccine-Data-by-County.xls


which is from: https://www.dshs.texas.gov/coronavirus/immunize/vaccine.aspx


Only Number, No Rate.


Three conceptions:

* Administered, 

* Vaccinated

* Fully Vaccinated

In [9]:
def get_Texas_Raw():
    TEXAS_URL = 'https://www.dshs.texas.gov/immunize/covid19/COVID-19-Vaccine-Data-by-County.xls'
    RawData = pd.read_excel(TEXAS_URL, sheet_name='By County, Race')
    return RawData


RawData = get_Texas_Raw()
RawData.columns = [i.strip() for i in RawData.columns]
RawData

,County Name,Race/Ethnicity,People Vaccinated with at least One Dose,People Fully Vaccinated,Doses Administered
0,Anderson,Asian,114,50,155
1,Anderson,Black,1220,612,1680
2,Anderson,Hispanic,983,352,1227
3,Anderson,Other,561,290,810
4,Anderson,Unknown,947,554,1367
...,...,...,...,...,...
1510,Other,Hispanic,24705,11528,34430
1511,Other,Other,7900,4323,11728
1512,Other,Unknown,13450,7561,20074
1513,Other,White,49374,25846,71469


In [10]:
def postprocess_Texas_Raw(RawData):
    RawDataGroups = RawData.groupby('County Name')
    L = []
    for county, record in RawDataGroups:
        D = {}

        D['County'] = county

        for idx, row in record.iterrows():
            d = row.to_dict()
            # print(d)
            race = d['Race/Ethnicity'] 

            D[race+'_DoesesAdmin'] = d['Doses Administered']
            D[race+'_Vaccinated']  = d['People Vaccinated with at least One Dose']
            D[race+'_FullVacc']     = d['People Fully Vaccinated']
        L.append(D)


    Texas_County_Vaccine_Race = pd.DataFrame(L)
    
    return Texas_County_Vaccine_Race


Texas_County_Vaccine_Race = postprocess_Texas_Raw(RawData)
# Texas_County_Vaccine_Race.to_csv('FinalResult/Post_Texas.csv')
print(Texas_County_Vaccine_Race.shape)
Texas_County_Vaccine_Race.head() # 254 counties + 1 total

(256, 22)


,County,Asian_DoesesAdmin,Asian_Vaccinated,Asian_FullVacc,Black_DoesesAdmin,Black_Vaccinated,Black_FullVacc,Hispanic_DoesesAdmin,Hispanic_Vaccinated,Hispanic_FullVacc,...,Other_FullVacc,Unknown_DoesesAdmin,Unknown_Vaccinated,Unknown_FullVacc,White_DoesesAdmin,White_Vaccinated,White_FullVacc,Total_DoesesAdmin,Total_Vaccinated,Total_FullVacc
0,Anderson,155.0,114.0,50.0,1680.0,1220.0,612.0,1227.0,983.0,352.0,...,290.0,1367.0,947.0,554.0,8936.0,5802.0,3859.0,NaN,NaN,NaN
1,Andrews,44.0,26.0,19.0,71.0,37.0,34.0,3673.0,1984.0,1700.0,...,91.0,2350.0,1333.0,1159.0,3785.0,2108.0,1712.0,NaN,NaN,NaN
2,Angelina,393.0,272.0,132.0,4125.0,2660.0,1662.0,5090.0,3691.0,1615.0,...,723.0,7355.0,3983.0,3444.0,18068.0,11428.0,7532.0,NaN,NaN,NaN
3,Aransas,258.0,179.0,86.0,82.0,54.0,31.0,2231.0,1513.0,845.0,...,828.0,2318.0,1258.0,1072.0,9585.0,6044.0,3911.0,NaN,NaN,NaN
4,Archer,24.0,20.0,13.0,69.0,65.0,36.0,172.0,126.0,81.0,...,395.0,1029.0,566.0,474.0,2705.0,1631.0,1302.0,NaN,NaN,NaN


In [11]:
import numpy as np
def standardize_Texas(Texas_County_Vaccine_Race, US_COUNTY_POPULATION):
    state = 'Texas'

    new_L = []


    for idx, row in Texas_County_Vaccine_Race.iterrows():
        d = row.to_dict()

        new_d = {}


        new_d['State'] = state
        new_d['County'] = d['County']

        RenameCounties = {
            # 'Mckean': 'McKean'
        }
        if new_d['County'] in RenameCounties:
            new_d['County'] = RenameCounties[new_d['County']]

        new_d['Geographic Area Name'] = new_d['County'] + ', ' + new_d['State']

        new_d['Numb_White'] = d['White_Vaccinated']
        new_d['Rate_White'] = np.nan

        new_d['Numb_Black'] = d['Black_Vaccinated']
        new_d['Rate_Black'] = np.nan

        new_d['Numb_Asian'] = d['Asian_Vaccinated']
        new_d['Rate_Asian'] = np.nan

        new_d['Numb_AIndA'] = np.nan
        new_d['Rate_AIndA'] = np.nan

        new_d['Numb_Other'] = d['Other_Vaccinated']
        new_d['Rate_Other'] = np.nan


        new_d['Numb_NaHis'] = np.nan
        new_d['Rate_NaHis'] = np.nan

        new_d['Numb_Hispa'] = d['Hispanic_Vaccinated']
        new_d['Rate_Hispa'] = np.nan



        new_d['Date'] = str(datetime.now().date())

        new_L.append(new_d)


    Data = pd.DataFrame(new_L)
    # StateCountyVaccine[state] = Data
    not_included = list(set(Data['Geographic Area Name'].unique()) - set(US_COUNTY_POPULATION['Geographic Area Name'].unique()))
    if len(not_included) > 0:
        print('Not included Counties', not_included)



    T = pd.merge(Data, US_COUNTY_POPULATION[US_COUNTY_POPULATION.columns[3:]], on = 'Geographic Area Name')


    for Race in ['White', 'Black', 'Asian', 'AIndA']:
        Rate = T['Numb_' + Race] / T['Total_' + Race]
        Rate = Rate.replace([-np.inf, np.inf], np.nan)
        T['Rate_' + Race] = Rate


    T = T[list(T.columns[:3]) + list(T.columns[3:].sort_values())]
    
    return T



T = standardize_Texas(Texas_County_Vaccine_Race, US_COUNTY_POPULATION)

state = 'Texas'
# T.to_csv('FinalResult/T_' + state +'.csv', index = False)
print(T.shape)
T.head()

Not included Counties ['Other, Texas', 'Grand Total, Texas']
(254, 27)


,State,County,Geographic Area Name,Date,Numb_AIndA,Numb_Asian,Numb_Black,Numb_Hispa,Numb_NaHis,Numb_Other,...,Rate_White,Total_AIndA,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole
0,Texas,Anderson,"Anderson, Texas",2021-04-17,NaN,114.0,1220.0,983.0,NaN,561.0,...,0.134820,428,747,538,757,12628,13200,43035,43927,57735
1,Texas,Andrews,"Andrews, Texas",2021-04-17,NaN,26.0,37.0,1984.0,NaN,121.0,...,0.119494,274,392,130,171,382,502,17641,17900,18705
2,Texas,Angelina,"Angelina, Texas",2021-04-17,NaN,272.0,2660.0,3691.0,NaN,1300.0,...,0.162732,665,1040,923,1166,13595,14256,70226,71385,86715
3,Texas,Aransas,"Aransas, Texas",2021-04-17,NaN,179.0,54.0,1513.0,NaN,1057.0,...,0.276727,308,542,491,564,406,554,21841,22252,23510
4,Texas,Archer,"Archer, Texas",2021-04-17,NaN,20.0,65.0,126.0,NaN,481.0,...,0.200418,131,200,41,66,96,153,8138,8264,8553


In [12]:
T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
# T['Rate_ParityC'] = T['Rate_WhiteC'] - T['Rate_BlackC']
STATE_VACCINE[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

['Illinois', 'Texas']


,name,count,mean,std,min,25%,50%,75%,max
0,Rate_Black,248.0,0.151325,0.174460,0.012821,0.076735,0.134183,0.181483,2.000000
1,Rate_White,254.0,0.157268,0.062251,0.019582,0.118887,0.159818,0.196356,0.413993
2,Rate_Parity,248.0,0.006849,0.164909,-1.844933,-0.017548,0.017462,0.056922,0.228222


## B. Pennsylvania* (Done)

67 counties

Ethnicity
https://data.pa.gov/Health/COVID-19-Vaccinations-by-Ethnicity-Current-County-/7ruj-m7k6 

Race
https://data.pa.gov/Health/COVID-19-Vaccinations-by-Race-Current-County-Healt/x5z9-57ub


Only Number

In [13]:

Penn_Ethn_URL = 'https://data.pa.gov/api/views/7ruj-m7k6/rows.csv?accessType=DOWNLOAD'
Penn_Race_URL = 'https://data.pa.gov/api/views/x5z9-57ub/rows.csv?accessType=DOWNLOAD'

Ethn = pd.read_csv(Penn_Ethn_URL).rename(columns = {'Ethnicity':'Item', 'County Name': 'County'})
Race = pd.read_csv(Penn_Race_URL).rename(columns = {'Race':'Item'})


RawData = pd.concat([Race, Ethn])

RawData.columns


Index(['County Name', 'Partially Covered African American',
       'Partially Covered Asian', 'Partially Covered Native American',
       'Partially Covered Pacific Islander',
       'Partially Covered Multiple Other', 'Partially Covered White',
       'Partially Covered  Unknown', 'Fully Covered African American',
       'Fully Covered Asian', 'Fully Covered Native American',
       'Fully Covered Pacific Islander', 'Fully Covered Multiple Other',
       'Fully Covered White', 'Fully Covered Unknown', 'County',
       'Partially Covered Hispanic', 'Partially Covered Not Hispanic',
       'Partially Covered Unknown', 'Fully Covered Hispanic',
       'Fully Covered Not Hispanic'],
      dtype='object')

In [14]:
Race.head()

,County Name,Partially Covered African American,Partially Covered Asian,Partially Covered Native American,Partially Covered Pacific Islander,Partially Covered Multiple Other,Partially Covered White,Partially Covered Unknown,Fully Covered African American,Fully Covered Asian,Fully Covered Native American,Fully Covered Pacific Islander,Fully Covered Multiple Other,Fully Covered White,Fully Covered Unknown
0,BRADFORD,20.0,NaN,NaN,NaN,68.0,4463,235,34.0,NaN,NaN,NaN,351,8650,444
1,MONTGOMERY,9709.0,1344.0,89.0,436.0,7107.0,149604,26525,6900.0,1203.0,73.0,289.0,13494,97871,24238
2,COLUMBIA,64.0,9.0,NaN,9.0,202.0,6538,264,93.0,7.0,NaN,15.0,940,15189,517
3,BEDFORD,15.0,9.0,NaN,8.0,117.0,4219,248,10.0,7.0,NaN,NaN,265,4726,901
4,VENANGO,26.0,NaN,NaN,NaN,246.0,2611,247,29.0,NaN,NaN,NaN,1511,6226,830


In [15]:
import pandas as pd

Pennsylvania_County_Vaccine_Race =  Race                         
Pennsylvania_County_Vaccine_Race

,County Name,Partially Covered African American,Partially Covered Asian,Partially Covered Native American,Partially Covered Pacific Islander,Partially Covered Multiple Other,Partially Covered White,Partially Covered Unknown,Fully Covered African American,Fully Covered Asian,Fully Covered Native American,Fully Covered Pacific Islander,Fully Covered Multiple Other,Fully Covered White,Fully Covered Unknown
0,BRADFORD,20.0,NaN,NaN,NaN,68.0,4463,235,34.0,NaN,NaN,NaN,351,8650,444
1,MONTGOMERY,9709.0,1344.0,89.0,436.0,7107.0,149604,26525,6900.0,1203.0,73.0,289.0,13494,97871,24238
2,COLUMBIA,64.0,9.0,NaN,9.0,202.0,6538,264,93.0,7.0,NaN,15.0,940,15189,517
3,BEDFORD,15.0,9.0,NaN,8.0,117.0,4219,248,10.0,7.0,NaN,NaN,265,4726,901
4,VENANGO,26.0,NaN,NaN,NaN,246.0,2611,247,29.0,NaN,NaN,NaN,1511,6226,830
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,CHESTER,3907.0,630.0,27.0,407.0,4551.0,86510,15705,3372.0,424.0,14.0,458.0,8246,91451,11312
64,CUMBERLAND,648.0,105.0,9.0,53.0,2782.0,25825,10718,1095.0,118.0,18.0,82.0,4976,37904,5247
65,CRAWFORD,215.0,NaN,NaN,NaN,213.0,5438,535,160.0,7.0,NaN,NaN,642,17771,1729
66,MERCER,215.0,NaN,NaN,5.0,485.0,5425,1308,951.0,6.0,NaN,5.0,2047,20413,4821


In [16]:
def standardize_Penn(Pennsylvania_County_Vaccine_Race, US_COUNTY_POPULATION):

    state = 'Pennsylvania' ## 

    new_L = []

    for idx, row in Pennsylvania_County_Vaccine_Race.iterrows(): ## 
        d = row.to_dict()

        new_d = {}
        new_d['State'] = state
        new_d['County'] = d['County Name'].lower().capitalize()

        RenameCounties = {
            'Mckean': 'McKean'
        }
        if new_d['County'] in RenameCounties:
            new_d['County'] = RenameCounties[new_d['County']]

        new_d['Geographic Area Name'] = new_d['County'] + ', ' + new_d['State']



        new_d['Numb_White'] = d['Partially Covered White']                  + d['Fully Covered White']   
        
        new_d['Numb_Black'] = d['Partially Covered African American']       + d['Fully Covered African American']
        
        new_d['Numb_Asian'] = d['Partially Covered Asian']                  + d['Fully Covered Asian']                  
         


        new_d['Date'] = str(datetime.now().date())

        new_L.append(new_d)


    Data = pd.DataFrame(new_L)
    
    not_included = list(set(Data['Geographic Area Name'].unique()) - set(US_COUNTY_POPULATION['Geographic Area Name'].unique()))
    if len(not_included) > 0:
        print('Not included Counties', not_included)

    
    T = pd.merge(Data, US_COUNTY_POPULATION[US_COUNTY_POPULATION.columns[3:]], on = 'Geographic Area Name')
    print(T.columns)
    for Race in ['White', 'Black', 'Asian']:
        Rate = T['Numb_' + Race] / T['Total_' + Race]
        Rate = Rate.replace([-np.inf, np.inf], np.nan)
        T['Rate_' + Race] = Rate



        
    T = T[list(T.columns[:3]) + list(T.columns[3:].sort_values())]
    
    return Data, T

In [17]:
Data, T = standardize_Penn(Pennsylvania_County_Vaccine_Race, US_COUNTY_POPULATION)
state = 'Pennsylvania'
# T.to_csv('FinalResult/T_' + state +'.csv', index = False)
print(T.shape)
T.head()

Not included Counties ['Out-of-state, Pennsylvania']
Index(['State', 'County', 'Geographic Area Name', 'Numb_White', 'Numb_Black',
       'Numb_Asian', 'Date', 'Total_Whole', 'Total_White', 'Total_Black',
       'Total_AIndA', 'Total_Asian', 'Total_WhiteC', 'Total_BlackC',
       'Total_AIndAC', 'Total_AsianC'],
      dtype='object')
(67, 19)


,State,County,Geographic Area Name,Date,Numb_Asian,Numb_Black,Numb_White,Rate_Asian,Rate_Black,Rate_White,Total_AIndA,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole
0,Pennsylvania,Bradford,"Bradford, Pennsylvania",2021-04-17,NaN,54.0,13113,NaN,0.110656,0.224284,183,514,397,566,488,791,58466,59198,60323
1,Pennsylvania,Montgomery,"Montgomery, Pennsylvania",2021-04-17,2547.0,16609.0,247475,0.037853,0.200099,0.375332,1977,5696,67286,73971,83004,94306,659350,676454,830915
2,Pennsylvania,Columbia,"Columbia, Pennsylvania",2021-04-17,16.0,157.0,21727,0.021277,0.119847,0.350849,163,422,752,938,1310,1681,61927,62642,64964
3,Pennsylvania,Bedford,"Bedford, Pennsylvania",2021-04-17,16.0,25.0,8945,0.075117,0.077882,0.191300,93,264,213,309,321,565,46759,47203,47888
4,Pennsylvania,Venango,"Venango, Pennsylvania",2021-04-17,NaN,55.0,8837,NaN,0.094664,0.180167,112,311,239,351,581,963,49049,49662,50668


In [18]:
T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
# T['Rate_ParityC'] = T['Rate_WhiteC'] - T['Rate_BlackC']
STATE_VACCINE[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

['Illinois', 'Texas', 'Pennsylvania']


,name,count,mean,std,min,25%,50%,75%,max
0,Rate_Black,60.0,0.146660,0.070983,0.018519,0.098634,0.133978,0.181827,0.384214
1,Rate_White,67.0,0.282471,0.078317,0.053800,0.236118,0.282544,0.336820,0.478258
2,Rate_Parity,60.0,0.132940,0.070998,-0.121782,0.095772,0.136036,0.172593,0.277505


In [19]:
T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
# T['Rate_ParityC'] = T['Rate_WhiteC'] - T['Rate_BlackC']
STATE_VACCINE[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

['Illinois', 'Texas', 'Pennsylvania']


,name,count,mean,std,min,25%,50%,75%,max
0,Rate_Black,60.0,0.146660,0.070983,0.018519,0.098634,0.133978,0.181827,0.384214
1,Rate_White,67.0,0.282471,0.078317,0.053800,0.236118,0.282544,0.336820,0.478258
2,Rate_Parity,60.0,0.132940,0.070998,-0.121782,0.095772,0.136036,0.172593,0.277505


## B. Indiana* (Done)

https://hub.mph.in.gov/dataset/covid-19-vaccinations-demographics-by-county-and-district/resource/82d99020-093f-41ac-95c7-d3c335b8c2ba

In [20]:
# path = 'TMP/Indiana_Mar26_county-vaccination-demographics.xlsx'
# path = 'TMP/Indiana/Indiana_April6_county-vaccination-demographics.xlsx'


url = 'https://hub.mph.in.gov/dataset/145a43b2-28e5-4bf1-ad86-123d07fddb55/resource/82d99020-093f-41ac-95c7-d3c335b8c2ba/download/county-vaccination-demographics.xlsx'

RaceData = pd.read_excel(url, sheet_name = 'Race').rename(columns = {"race":"item"})
EthnicityData = pd.read_excel(url, sheet_name = 'Ethnicity').rename(columns = {"ethnicity":"item"})
Data = pd.concat([RaceData, EthnicityData])
Data.head()

,county,region,fips,item,first_dose_administered,second_dose_administered,single_dose_administered,all_doses_administered,fully_vaccinated,new_first_dose_administered,new_second_dose_administered,new_single_dose_administered,new_all_doses_administered,new_fully_vaccinated,current_as_of
0,Adams,3,18001,Asian,38,21,Suppressed,59,21,Suppressed,Suppressed,Suppressed,Suppressed,Suppressed,4/17/2021 5:00
1,Adams,3,18001,Black or African American,14,6,Suppressed,24,10,Suppressed,Suppressed,Suppressed,Suppressed,Suppressed,4/17/2021 5:00
2,Adams,3,18001,Other Race,165,112,33,310,145,Suppressed,Suppressed,Suppressed,Suppressed,Suppressed,4/17/2021 5:00
3,Adams,3,18001,Unknown,165,91,157,413,248,Suppressed,Suppressed,Suppressed,Suppressed,Suppressed,4/17/2021 5:00
4,Adams,3,18001,White,7727,6125,336,14188,6461,40,98,Suppressed,138,98,4/17/2021 5:00


In [21]:

# Data.columns


L= []

for county, record in Data.groupby('county'):
    # print(county)
    D = {}
    D['county'] = county

    for idx, row in record.iterrows():
        d = row.to_dict()
        # print(d)
        race = d['item']
        try:
            D[race + '_' + 'AtLeastOneDose'] = float(d['all_doses_administered'])
        except:
            D[race + '_' + 'AtLeastOneDose'] = np.nan
    # print(D)
    L.append(D)

Indiana_County_Vaccine_Race = pd.DataFrame(L)



In [22]:
print(Indiana_County_Vaccine_Race.shape)
Indiana_County_Vaccine_Race.head() # 95 counties in Pennsylvania

(94, 8)


,county,Asian_AtLeastOneDose,Black or African American_AtLeastOneDose,Other Race_AtLeastOneDose,Unknown_AtLeastOneDose,White_AtLeastOneDose,Hispanic or Latino_AtLeastOneDose,Not Hispanic or Latino_AtLeastOneDose
0,Adams,59.0,24.0,310.0,1035.0,14188.0,371.0,13588.0
1,Allen,5990.0,12420.0,8803.0,24054.0,160096.0,5556.0,165630.0
2,Bartholomew,2685.0,595.0,1312.0,1774.0,42481.0,1344.0,45412.0
3,Benton,9.0,16.0,106.0,172.0,3301.0,136.0,3221.0
4,Blackford,10.0,6.0,52.0,243.0,5845.0,38.0,5766.0


In [23]:
# Indiana_County_Vaccine_Race.to_csv('FinalResult/Indiana.csv')
print(Indiana_County_Vaccine_Race.shape)
Indiana_County_Vaccine_Race.head() # 95 counties in Pennsylvania

(94, 8)


,county,Asian_AtLeastOneDose,Black or African American_AtLeastOneDose,Other Race_AtLeastOneDose,Unknown_AtLeastOneDose,White_AtLeastOneDose,Hispanic or Latino_AtLeastOneDose,Not Hispanic or Latino_AtLeastOneDose
0,Adams,59.0,24.0,310.0,1035.0,14188.0,371.0,13588.0
1,Allen,5990.0,12420.0,8803.0,24054.0,160096.0,5556.0,165630.0
2,Bartholomew,2685.0,595.0,1312.0,1774.0,42481.0,1344.0,45412.0
3,Benton,9.0,16.0,106.0,172.0,3301.0,136.0,3221.0
4,Blackford,10.0,6.0,52.0,243.0,5845.0,38.0,5766.0


In [24]:
# print(len(L))

import numpy as np



state = 'Indiana' ## 

new_L = []


for idx, row in Indiana_County_Vaccine_Race.iterrows(): ## 
    d = row.to_dict()
    # print(d)
    new_d = {}
    
    if type(d['county']) != str:
        print(d)
        continue 
        
    new_d['State'] = state
    new_d['County'] = ' '.join([i.lower().capitalize() for i in d['county'].split(' ')])
    RenameCounties = {
        # 'Mckean': 'McKean'
        'Lagrange': 'LaGrange',
        'La Porte': 'LaPorte', 
        'De Kalb': 'DeKalb',
        
    }
    if new_d['County'] in RenameCounties:
        new_d['County'] = RenameCounties[new_d['County']]

    new_d['Geographic Area Name'] = new_d['County'] + ', ' + new_d['State']

    new_d['Numb_White'] = float(d['White_AtLeastOneDose'])
    new_d['Rate_White'] = np.nan
    
    new_d['Numb_Black'] = d['Black or African American_AtLeastOneDose']
    new_d['Rate_Black'] = np.nan
    
    new_d['Numb_Asian'] = d['Asian_AtLeastOneDose']
    new_d['Rate_Asian'] = np.nan
    
    new_d['Numb_AIndA'] = np.nan 
    new_d['Rate_AIndA'] = np.nan
    
    new_d['Numb_Other'] = np.nan
    new_d['Rate_Other'] = np.nan
    
    
    new_d['Numb_NaHis'] = d['Not Hispanic or Latino_AtLeastOneDose']
    new_d['Rate_NaHis'] = np.nan
    
    new_d['Numb_Hispa'] = d['Hispanic or Latino_AtLeastOneDose']
    new_d['Rate_Hispa'] = np.nan
    
    new_d['Date'] = np.nan
    
    new_L.append(new_d)
    
    
Data = pd.DataFrame(new_L)

not_included = list(set(Data['Geographic Area Name'].unique()) - set(US_COUNTY_POPULATION['Geographic Area Name'].unique()))
if len(not_included) > 0:
    print('Not included Counties', not_included)


T = pd.merge(Data, US_COUNTY_POPULATION[US_COUNTY_POPULATION.columns[3:]], on = 'Geographic Area Name')


for Race in ['White', 'Black', 'Asian']:
    Rate = T['Numb_' + Race] / T['Total_' + Race]
    Rate = Rate.replace([-np.inf, np.inf], np.nan)
    T['Rate_' + Race] = Rate

    Rate = T['Numb_' + Race] / T['Total_' + Race + 'C']
    Rate = Rate.replace([-np.inf, np.inf], np.nan)
    T['Rate_' + Race + 'C'] = Rate


T = T[list(T.columns[:3]) + list(T.columns[3:].sort_values())]
print(T.shape)
T.head()


Not included Counties ['Unknown, Indiana', 'Out Of State, Indiana']
(92, 30)


,State,County,Geographic Area Name,Date,Numb_AIndA,Numb_Asian,Numb_Black,Numb_Hispa,Numb_NaHis,Numb_Other,...,Rate_WhiteC,Total_AIndA,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole
0,Indiana,Adams,"Adams, Indiana",NaN,NaN,59.0,24.0,371.0,13588.0,NaN,...,0.401926,117,208,137,183,198,384,34995,35300,35777
1,Indiana,Allen,"Allen, Indiana",NaN,NaN,5990.0,12420.0,5556.0,165630.0,NaN,...,0.510502,1780,4346,17431,19785,45450,53961,302000,313605,379299
2,Indiana,Bartholomew,"Bartholomew, Indiana",NaN,NaN,2685.0,595.0,1344.0,45412.0,NaN,...,0.572929,390,806,7133,7427,1974,2756,72810,74147,83779
3,Indiana,Benton,"Benton, Indiana",NaN,NaN,9.0,16.0,136.0,3221.0,NaN,...,0.384061,28,69,21,31,97,165,8479,8595,8748
4,Indiana,Blackford,"Blackford, Indiana",NaN,NaN,10.0,6.0,38.0,5766.0,NaN,...,0.505623,31,111,67,98,90,172,11378,11560,11758


In [25]:
T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
# T['Rate_ParityC'] = T['Rate_WhiteC'] - T['Rate_BlackC']
# STATE_VACCINE = {}
STATE_VACCINE[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

T

['Illinois', 'Texas', 'Pennsylvania', 'Indiana']


,State,County,Geographic Area Name,Date,Numb_AIndA,Numb_Asian,Numb_Black,Numb_Hispa,Numb_NaHis,Numb_Other,...,Total_AIndA,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole,Rate_Parity
0,Indiana,Adams,"Adams, Indiana",NaN,NaN,59.0,24.0,371.0,13588.0,NaN,...,117,208,137,183,198,384,34995,35300,35777,0.284217
1,Indiana,Allen,"Allen, Indiana",NaN,NaN,5990.0,12420.0,5556.0,165630.0,NaN,...,1780,4346,17431,19785,45450,53961,302000,313605,379299,0.256852
2,Indiana,Bartholomew,"Bartholomew, Indiana",NaN,NaN,2685.0,595.0,1344.0,45412.0,NaN,...,390,806,7133,7427,1974,2756,72810,74147,83779,0.282032
3,Indiana,Benton,"Benton, Indiana",NaN,NaN,9.0,16.0,136.0,3221.0,NaN,...,28,69,21,31,97,165,8479,8595,8748,0.224366
4,Indiana,Blackford,"Blackford, Indiana",NaN,NaN,10.0,6.0,38.0,5766.0,NaN,...,31,111,67,98,90,172,11378,11560,11758,0.447044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,Indiana,Washington,"Washington, Indiana",NaN,NaN,34.0,23.0,68.0,11439.0,NaN,...,77,208,85,157,143,246,27434,27718,28036,0.269320
88,Indiana,Wayne,"Wayne, Indiana",NaN,NaN,402.0,1332.0,605.0,33841.0,NaN,...,252,673,713,1041,3316,4618,59598,61394,65884,0.138229
89,Indiana,Wells,"Wells, Indiana",NaN,NaN,59.0,34.0,144.0,12192.0,NaN,...,84,201,189,248,249,429,27415,27754,28296,0.330534
90,Indiana,White,"White, Indiana",NaN,NaN,44.0,23.0,335.0,13511.0,NaN,...,176,327,136,202,217,333,23236,23549,24102,0.490454


In [26]:
T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
# T['Rate_ParityC'] = T['Rate_WhiteC'] - T['Rate_BlackC']
STATE_VACCINE[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

['Illinois', 'Texas', 'Pennsylvania', 'Indiana']


,name,count,mean,std,min,25%,50%,75%,max
0,Rate_Black,90.0,0.250984,0.153383,0.040268,0.151710,0.232457,0.309762,1.146214
1,Rate_White,92.0,0.509758,0.100411,0.262534,0.436674,0.514197,0.572219,0.815487
2,Rate_Parity,90.0,0.261804,0.135086,-0.646353,0.212444,0.262049,0.329613,0.514514


## B. Virginia* (Done)

https://data.virginia.gov/Government/VDH-COVID-19-PublicUseDataset-Vaccines-DosesAdmini/u5ru-3khs

In [27]:
import requests
from  sodapy import Socrata


# Today_Date = str(datetime.now().date())


##########################################
report_date = Today_Date + 'T00:00:00.000'
##########################################

client = Socrata("data.virginia.gov", None)

url = 'https://data.virginia.gov/resource/u5ru-3khs.json?report_date=' + report_date + 'T00:00:00.000?limit=3000'

results = client.get("u5ru-3khs", limit=5000, report_date = report_date)

# Convert to pandas DataFrame
RawData = pd.DataFrame.from_records(results)

In [28]:
while len(RawData) == 0:
    print('Error in Date, Please Change Date:')
    ##########################################
    new_date_for_virgina = input()
    report_date = new_date_for_virgina + 'T00:00:00.000'
    ##########################################

    client = Socrata("data.virginia.gov", None)

    url = 'https://data.virginia.gov/resource/u5ru-3khs.json?report_date=' + report_date + 'T00:00:00.000?limit=3000'

    results = client.get("u5ru-3khs", limit=5000, report_date = report_date)

    # Convert to pandas DataFrame
    RawData = pd.DataFrame.from_records(results)
    
RawData.head()

,report_date,fips,locality,health_district,health_region,vaccination_status,race_and_ethnicity,people_by_vaccination_status_count
0,2021-04-17T00:00:00.000,51195,Wise,Lenowisco,Southwest,Not Reported,Native American,0
1,2021-04-17T00:00:00.000,51570,Colonial Heights,Chesterfield,Central,Fully Vaccinated,White,2015
2,2021-04-17T00:00:00.000,51135,Nottoway,Piedmont,Central,Not Reported,Two or more races,0
3,2021-04-17T00:00:00.000,51153,Prince William,Prince William,Northern,Not Reported,Black,0
4,2021-04-17T00:00:00.000,51113,Madison,Rappahannock Rapidan,Northwest,Not Reported,Two or more races,0


In [29]:
RawDataGroups = RawData.groupby('locality')


L = []
for county, record in RawDataGroups:
    D = {}

    D['County'] = county

    for idx, row in record.iterrows():
        d = row.to_dict()
        race = d['race_and_ethnicity'] 
        
        d = row.to_dict()
        # print(d)
        D[race + '_' + d['vaccination_status']] = float(d['people_by_vaccination_status_count'])
        

    L.append(D)

    
Virginia_County_Vaccine_Race = pd.DataFrame(L)

In [30]:
# Virginia_County_Vaccine_Race.to_csv('FinalResult/Vriginia.csv')
print(Virginia_County_Vaccine_Race.shape)
Virginia_County_Vaccine_Race.head() # 67 counties in Pennsylvania

(135, 25)


,County,Not Reported_Fully Vaccinated,Two or more races_At Least One Dose,Two or more races_Fully Vaccinated,Native American_Not Reported,White_Fully Vaccinated,Asian or Pacific Islander_Fully Vaccinated,Black_Fully Vaccinated,Latino_Fully Vaccinated,Asian or Pacific Islander_Not Reported,...,Black_At Least One Dose,Latino_At Least One Dose,White_At Least One Dose,Two or more races_Not Reported,Native American_Fully Vaccinated,Not Reported_Not Reported,Not Reported_At Least One Dose,Other Race_Fully Vaccinated,Other Race_At Least One Dose,Asian or Pacific Islander_At Least One Dose
0,Accomack,1766.0,0.0,0.0,0.0,5410.0,53.0,1514.0,300.0,0.0,...,2055.0,507.0,6539.0,0.0,16.0,0.0,2887.0,21.0,31.0,87.0
1,Albemarle,6972.0,0.0,0.0,0.0,21379.0,647.0,1923.0,1062.0,0.0,...,2929.0,2261.0,31841.0,0.0,109.0,0.0,15930.0,1530.0,2380.0,1105.0
2,Alexandria,15071.0,0.0,0.0,0.0,10691.0,760.0,2193.0,3280.0,0.0,...,3458.0,6120.0,15270.0,0.0,89.0,0.0,28047.0,2077.0,2980.0,1329.0
3,Alleghany,1232.0,0.0,0.0,0.0,1879.0,11.0,99.0,31.0,0.0,...,177.0,83.0,2925.0,0.0,5.0,0.0,2433.0,37.0,67.0,19.0
4,Amelia,574.0,0.0,0.0,0.0,1361.0,13.0,537.0,34.0,0.0,...,748.0,63.0,1912.0,0.0,11.0,0.0,984.0,21.0,35.0,22.0


In [31]:
# Virginia_County_Vaccine_Race.to_csv('FinalResult/Vriginia.csv')
print(Virginia_County_Vaccine_Race.shape)
Virginia_County_Vaccine_Race.head() # 67 counties in Pennsylvania

(135, 25)


,County,Not Reported_Fully Vaccinated,Two or more races_At Least One Dose,Two or more races_Fully Vaccinated,Native American_Not Reported,White_Fully Vaccinated,Asian or Pacific Islander_Fully Vaccinated,Black_Fully Vaccinated,Latino_Fully Vaccinated,Asian or Pacific Islander_Not Reported,...,Black_At Least One Dose,Latino_At Least One Dose,White_At Least One Dose,Two or more races_Not Reported,Native American_Fully Vaccinated,Not Reported_Not Reported,Not Reported_At Least One Dose,Other Race_Fully Vaccinated,Other Race_At Least One Dose,Asian or Pacific Islander_At Least One Dose
0,Accomack,1766.0,0.0,0.0,0.0,5410.0,53.0,1514.0,300.0,0.0,...,2055.0,507.0,6539.0,0.0,16.0,0.0,2887.0,21.0,31.0,87.0
1,Albemarle,6972.0,0.0,0.0,0.0,21379.0,647.0,1923.0,1062.0,0.0,...,2929.0,2261.0,31841.0,0.0,109.0,0.0,15930.0,1530.0,2380.0,1105.0
2,Alexandria,15071.0,0.0,0.0,0.0,10691.0,760.0,2193.0,3280.0,0.0,...,3458.0,6120.0,15270.0,0.0,89.0,0.0,28047.0,2077.0,2980.0,1329.0
3,Alleghany,1232.0,0.0,0.0,0.0,1879.0,11.0,99.0,31.0,0.0,...,177.0,83.0,2925.0,0.0,5.0,0.0,2433.0,37.0,67.0,19.0
4,Amelia,574.0,0.0,0.0,0.0,1361.0,13.0,537.0,34.0,0.0,...,748.0,63.0,1912.0,0.0,11.0,0.0,984.0,21.0,35.0,22.0


In [32]:
Virginia_County_Vaccine_Race['County'].values

array(['Accomack', 'Albemarle', 'Alexandria', 'Alleghany', 'Amelia',
       'Amherst', 'Appomattox', 'Arlington', 'Augusta', 'Bath', 'Bedford',
       'Bland', 'Botetourt', 'Bristol', 'Brunswick', 'Buchanan',
       'Buckingham', 'Buena Vista City', 'Campbell', 'Caroline',
       'Carroll', 'Charles City', 'Charlotte', 'Charlottesville',
       'Chesapeake', 'Chesterfield', 'Clarke', 'Colonial Heights',
       'Covington', 'Craig', 'Culpeper', 'Cumberland', 'Danville',
       'Dickenson', 'Dinwiddie', 'Emporia', 'Essex', 'Fairfax',
       'Fairfax City', 'Falls Church', 'Fauquier', 'Floyd', 'Fluvanna',
       'Franklin City', 'Franklin County', 'Frederick', 'Fredericksburg',
       'Galax', 'Giles', 'Gloucester', 'Goochland', 'Grayson', 'Greene',
       'Greensville', 'Halifax', 'Hampton', 'Hanover', 'Harrisonburg',
       'Henrico', 'Henry', 'Highland', 'Hopewell', 'Isle of Wight',
       'James City', 'King George', 'King William', 'King and Queen',
       'Lancaster', 'Lee', 'Lexing

In [33]:
# print(len(L))

import numpy as np


virginia_name_dict = {
 'Colonial Heights, Virginia': ['Colonial Heights city, Virginia'],
 
 'Portsmouth, Virginia': ['Portsmouth city, Virginia'],
    
 'Roanoke City, Virginia': ['Roanoke city, Virginia'],
    
 'Charlottesville, Virginia': ['Charlottesville city, Virginia'],
    
 'Waynesboro, Virginia': ['Waynesboro city, Virginia'],
    
 'Virginia Beach, Virginia': ['Virginia Beach city, Virginia'],
    
 'Suffolk, Virginia': ['Suffolk city, Virginia'],
    
 'Williamsburg, Virginia': ['Williamsburg city, Virginia'],
    
 'Roanoke County, Virginia': ['Roanoke, Virginia'],
    
 'Falls Church, Virginia': ['Falls Church city, Virginia'],
    
 'Franklin County, Virginia': ['Franklin, Virginia'],
    
 'Buena Vista City, Virginia': ['Buena Vista city, Virginia'],
 
 'Lynchburg, Virginia': ['Lynchburg city, Virginia'],
    
 'Hampton, Virginia': ['Hampton city, Virginia'],
    
 'Fredericksburg, Virginia': ['Fredericksburg city, Virginia'],
    
 'Staunton, Virginia': ['Staunton city, Virginia'],
    
 'Galax, Virginia': ['Galax city, Virginia'],
    
 'Richmond County, Virginia': ['Richmond, Virginia'],
    
 'Petersburg, Virginia': ['Petersburg city, Virginia'],
    
 'Harrisonburg, Virginia': ['Harrisonburg city, Virginia'],
    
 'Richmond City, Virginia': ['Richmond city, Virginia'],
    
 'Covington, Virginia': ['Covington city, Virginia'],
    
 'Norfolk, Virginia': ['Norfolk city, Virginia'],
    
 'Fairfax City, Virginia': ['Fairfax city, Virginia'],
    
 'Bristol, Virginia': ['Bristol city, Virginia'],
    
 'Isle Of Wight, Virginia': ['Isle of Wight, Virginia'],
    
 'Lexington, Virginia': ['Lexington city, Virginia'],
    
 'Hopewell, Virginia': ['Hopewell city, Virginia'],
    
 'Alexandria, Virginia': ['Alexandria city, Virginia'],
    
 'Manassas City, Virginia': ['Manassas city, Virginia'],
    
 'Winchester, Virginia': ['Winchester city, Virginia'],
    
 'Manassas Park, Virginia': ['Manassas Park city, Virginia'],
    
 'Poquoson, Virginia': ['Poquoson city, Virginia'],
    
 'Emporia, Virginia': ['Emporia city, Virginia'],
    
 'Radford, Virginia': ['Radford city, Virginia'],
    
 'Salem, Virginia': ['Salem city, Virginia'],
    
 'Chesapeake, Virginia': ['Chesapeake city, Virginia'],
    
 'King And Queen, Virginia': ['King and Queen, Virginia'],
    
 'Franklin City, Virginia': ['Franklin city, Virginia'],
    
 'Norton, Virginia': ['Norton city, Virginia'],
  
 'Danville, Virginia': ['Danville city, Virginia'],
    
 'Newport News, Virginia': ['Newport News city, Virginia'],
    
 'Martinsville, Virginia': ['Martinsville city, Virginia']}

RenameCounties = {k.replace(', Virginia', ''): v[0].replace(', Virginia', '') for k, v in virginia_name_dict.items()}

state = 'Virginia' ## 

new_L = []


for idx, row in Virginia_County_Vaccine_Race.iterrows(): ## 
    d = row.to_dict()
    # print(d)
    new_d = {}
    
    if type(d['County']) != str:
        print(d)
        continue 
        
    new_d['State'] = state
    new_d['County'] = ' '.join([i.lower().capitalize() for i in d['County'].split(' ')])
    if new_d['County'] in RenameCounties:
        new_d['County'] = RenameCounties[new_d['County']]

    new_d['Geographic Area Name'] = new_d['County'] + ', ' + new_d['State']

    
    new_d['Numb_White'] = d['White_At Least One Dose']
    new_d['Rate_White'] = np.nan
    
    new_d['Numb_Black'] = d['Black_At Least One Dose']
    new_d['Rate_Black'] = np.nan
    
    new_d['Numb_Asian'] = d['Asian or Pacific Islander_At Least One Dose']
    
    new_d['Date'] = np.nan
    
    new_L.append(new_d)
    
    
Data = pd.DataFrame(new_L)


not_included = list(set(Data['Geographic Area Name'].unique()) - set(US_COUNTY_POPULATION['Geographic Area Name'].unique()))
if len(not_included) > 0:
    print('Not included Counties', not_included)


T = pd.merge(Data, US_COUNTY_POPULATION[US_COUNTY_POPULATION.columns[3:]], on = 'Geographic Area Name')


for Race in ['White', 'Black', 'Asian']:
    Rate = T['Numb_' + Race] / T['Total_' + Race]
    Rate = Rate.replace([-np.inf, np.inf], np.nan)
    T['Rate_' + Race] = Rate

#     Rate = T['Numb_' + Race] / T['Total_' + Race + 'C']
#     Rate = Rate.replace([-np.inf, np.inf], np.nan)
#     T['Rate_' + Race + 'C'] = Rate


T = T[list(T.columns[:3]) + list(T.columns[3:].sort_values())]
print(T.shape)
T.head()

    

Not included Counties ['Out-of-state, Virginia', 'Not Reported, Virginia']
(133, 19)


,State,County,Geographic Area Name,Date,Numb_Asian,Numb_Black,Numb_White,Rate_Asian,Rate_Black,Rate_White,Total_AIndA,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole
0,Virginia,Accomack,"Accomack, Virginia",NaN,87.0,2055.0,6539.0,0.338521,0.220873,0.298598,239,447,257,348,9304,9632,21899,22411,32316
1,Virginia,Albemarle,"Albemarle, Virginia",NaN,1105.0,2929.0,31841.0,0.182614,0.276321,0.356211,383,1000,6051,7189,10600,12008,89388,92002,109330
2,Virginia,Alexandria city,"Alexandria city, Virginia",NaN,1329.0,3458.0,15270.0,0.128059,0.095288,0.143611,998,2243,10378,12886,36290,38626,106329,110857,159428
3,Virginia,Alleghany,"Alleghany, Virginia",NaN,19.0,177.0,2925.0,0.413043,0.253582,0.212218,44,119,46,74,698,890,13783,14040,14860
4,Virginia,Amelia,"Amelia, Virginia",NaN,22.0,748.0,1912.0,0.275000,0.278274,0.190249,79,161,80,124,2688,2827,10050,10264,13145


In [34]:
# Virginia	Accomack	Accomack, Virginia	NaN	63.0	1704.0	5764.0	0.245136	0.183147	0.263208	239	447	257
T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
# T['Rate_ParityC'] = T['Rate_WhiteC'] - T['Rate_BlackC']
STATE_VACCINE[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

['Illinois', 'Texas', 'Pennsylvania', 'Indiana', 'Virginia']


,name,count,mean,std,min,25%,50%,75%,max
0,Rate_Black,133.0,0.195879,0.082984,0.060780,0.133624,0.176449,0.232627,0.565217
1,Rate_White,133.0,0.217026,0.064989,0.082753,0.167851,0.208196,0.253449,0.451538
2,Rate_Parity,133.0,0.021147,0.081486,-0.399081,-0.009060,0.029922,0.064155,0.252923


In [35]:
T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
# T['Rate_ParityC'] = T['Rate_WhiteC'] - T['Rate_BlackC']
STATE_VACCINE[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

['Illinois', 'Texas', 'Pennsylvania', 'Indiana', 'Virginia']


,name,count,mean,std,min,25%,50%,75%,max
0,Rate_Black,133.0,0.195879,0.082984,0.060780,0.133624,0.176449,0.232627,0.565217
1,Rate_White,133.0,0.217026,0.064989,0.082753,0.167851,0.208196,0.253449,0.451538
2,Rate_Parity,133.0,0.021147,0.081486,-0.399081,-0.009060,0.029922,0.064155,0.252923


In [36]:
# T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
# # T['Rate_ParityC'] = T['Rate_WhiteC'] - T['Rate_BlackC']
# STATE_VACCINE[state] = T
# print([i for i in STATE_VACCINE])
# diagnose_Rate(T)

# All in One

## Read Previous Vaccine Data

In [37]:
# import os
# path = ''

# print('Input Historical STATE_VACCINE')
# print('For example: RaceVaccineBackup/STATE_VACCINE_2021-03-23.p')

# L = []
# while True:
    
#     path = input()
    
#     if os.path.isfile(path):
#         print('Path Exists!')
#         L.append(path)
#     elif path != '':
#         print('Wrong Path')
#     else:
#         break

# print('End')

In [38]:
# import pickle

# Historical_Path = {i.split('_')[-1].replace('.p', ''): i for i in L }

# Historical_Data = {}
# for date, path in Historical_Path.items():
#     with open(path, 'rb' ) as f:
#         Historical_Data[date] = pickle.load(f)
    
    
# [i for i in Historical_Data]

## Show Parity for Each State

In [39]:
# ######################################

# # Today_Date = str(datetime.now().date())
# print(Today_Date)
# ######################################


# for state in STATE_VACCINE:
    
#     print(state)
#     cols = ['name',  'date', 'count', 'mean', 'min','50%', 'max']
    
    
    
#     for date, Old_STATE_VACCINE in Historical_Data.items():
#         x = diagnose_Rate(Old_STATE_VACCINE[state], state = state, date = date)
#         print(x[cols])
    
#     x = diagnose_Rate(STATE_VACCINE[state], state = state, date = Today_Date)
    
#     print(x[cols])
#     print('\n')
    
    
# # pd.concat(L).to_csv('compare.csv')

##  Show Some Specials Cases

In [40]:

# DF = pd.concat([v for k,v in STATE_VACCINE.items()]) 

# data_cols = ['State', 'County', 'Geographic Area Name',
#              'Total_Black', 'Total_White',
#              'Numb_Black', 'Numb_White','Rate_Black','Rate_White', 'Total_Whole', 'Rate_Parity']


# DF[DF['Rate_Black'] > 0.8][data_cols]

## Save Data

In [41]:

import pickle 

path = 'Workplace/STATE_VACCINE_' + Today_Date + '_Autonomy.p'

while True:
    print('The data will be saved here, do you agree?' )
    print(path)
    print('If yes, please enter: Yes')
    x = input()
    if x =='Yes':
        break


with open(path, 'wb') as f:
    pickle.dump(STATE_VACCINE, f)
    
    
print('Successfully Saved!')

The data will be saved here, do you agree?
Workplace/STATE_VACCINE_2021-04-17_Autonomy.p
If yes, please enter: Yes
Yes
Successfully Saved!
